In [31]:
#Import libraries

import pandas as pd
from bs4 import BeautifulSoup
import re
import json
from pdfminer.high_level import extract_text
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextBoxHorizontal
import os
import requests
from urllib.parse import urlparse
import numpy as np
# import nltk
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer


## <h3> Open Metadata </h3>

In [9]:
#Load Metadata CSV
metadata_df = pd.read_excel('../01_Input/00_Metadata/National Energy Document Compilation.xlsx')
metadata_df=metadata_df[metadata_df["Exists?"]=="Y"]

# metadata_df=metadata_df.head(50)
#'../01_Input/00_Metadata/seh-document-metadata.csv')

#SK: testset
metadata_df_test=metadata_df[metadata_df["Country Code"]=="LKA"]
metadata_df_test.head(5)

,Code,Status,Country Name,Country Code,Category,KeyWord to Search,Title,Exists?,Format,Publication Date,Publication Year,Start Year,End Year,Language,Link
3586,LKA-CPD-2022-EN,Completed,Sri Lanka,LKA,CPD,NaN,Country programme document for Sri Lanka (2023...,Y,Text,29 August–1 September 2022,2022,2023,2027,EN,https://digitallibrary.un.org/record/3982477/f...
3587,LKA-CPD-2022-FR,NaN,Sri Lanka,LKA,CPD,NaN,NaN,Y,Text,29 August–1 September 2022,2022,2023,2027,FR,https://digitallibrary.un.org/record/3982477/f...
3588,LKA-CPD-2022-SP,NaN,Sri Lanka,LKA,CPD,NaN,NaN,Y,Text,29 August–1 September 2022,2022,2023,2027,SP,https://digitallibrary.un.org/record/3982477/f...
3589,LKA-NEP-2008-EN,Completed,Sri Lanka,LKA,NEP,NaN,NATIONAL ENERGY POLICY & STRATEGIES OF SRI LANKA,Y,Text,2008,2008,2008,2017,EN,https://policy.asiapacificenergy.org/sites/def...
3590,LKA-NEP-2019-EN,Completed,Sri Lanka,LKA,NEP,NaN,NATIONAL ENERGY POLICY AND STRATEGIES OF SRI L...,Y,Text,2019,2019,2019,2030,EN,https://policy.asiapacificenergy.org/sites/def...


## <h3> Donwload PDFs </h3>

In [7]:
#download a pdf from a url to a path if is exists
def download_document(pdf_url,pdf_path):
    # Code to download the document from the URL
    # Handle different formats (PDF, HTML, etc.)
    try:
        response = requests.get(pdf_url,timeout=8)

        if response.status_code == 200:
            #content_disposition = response.headers.get("content-disposition")
            with open(pdf_path, "wb") as file:
                file.write(response.content)
            print(f"{pdf_path} successfully downloaded document with path: {pdf_url}")
            return pdf_path
        else:
            print(f"Code 200, failed to downloaded document with path: {pdf_url}")
            return "None"
    except:
        print(f"Failure to download document with path: {pdf_url}")
        return "None"
#Example Usage


In [8]:
#Download missing document
def download_document_if_missing(row,pdf_path):

    if not os.path.exists(pdf_path):
        ##Download document if the pdf is not already present
        pdf_url = row['Link']

        if not pd.isna(pdf_url):
            download_document(pdf_url,pdf_path)
        else:
            print(pdf_path,"no pdf or link")
    else:
        print(pdf_path,"already exists")
#Example Usage


In [16]:
#Run the download function for all
def download_all(metadata_df, missing_only=True):

    for index, row in metadata_df.iterrows():
        folder="../01_Input/01_PDFs/03_National/"+row["Code"].split("-")[0]

        # Check if the directory already exists
        if not os.path.exists(folder):
            # Create the directory
            os.makedirs(folder)
            print(folder+"directory created!")

        pdf_path=folder+'/'+row["Code"]+".pdf"
        download_document_if_missing(row, pdf_path)

In [ ]:
## Run the download pipeline for all
# download_all(metadata_df,missing_only=True)

## <h3> Text Extraction </h3>

In [1]:
#Extract text from document
def extract_text_from_pdf(extract_text_path, pdf_path):

    try:
        text = extract_text(pdf_path)##imported function
        ##export text
        with open(extract_text_path, 'w') as file:
            file.write(text)
    except:
        print("invalid pdf",pdf_path)


In [41]:
test_pdf = "../01_Input/01_PDFs/03_National/LKA/LKA-CPD-2022-EN.pdf"
test_pdf2 = "../01_Input/01_PDFs/03_National/LKA/LKA-NEPro-2022-EN.pdf"
# text = extract_text(test_pdf)
test_extract_path = "../02_Output/00_Extracted-Text/LKA_test.txt"

In [56]:
text_ext = []
for page_layout in extract_pages(test_pdf):
    for element in page_layout:

        if isinstance(element, LTTextBoxHorizontal):
            if len(element.get_text().split()) != 1 and '…' not in element.get_text() and '...' not in element.get_text():
                text_ext.append(element.get_text().replace('\n', '').replace('  ', ''))

with open(test_extract_path, 'w') as file:
    file.write("\n".join(text_ext))
# text_ext


In [2]:
#Download missing documents
def extract_text_if_missing(extract_text_path, pdf_path):

    if not os.path.exists(extract_text_path):
        ##extract text if it is not already present
        extract_text_from_pdf(extract_text_path, pdf_path)
    else:
        print(extract_text_path,"already exists")

In [4]:
#Run the extract function for all
def extract_all(metadata_df, missing_only=True):

    for index, row in metadata_df.iterrows():
        # folder="03_National/"+row["Code"].split("-")[0]## update this for more docs
        folder="03_National/LKA"## update this for more docs
        pdf_path="../01_Input/01_PDFs/"+folder+'/'+row["Code"]+".pdf"
        extract_text_folder="../02_Output/00_Extracted-Text/"+folder+'2'

        if os.path.exists(pdf_path):

            if not os.path.exists(extract_text_folder):
                # Create the directory
                os.makedirs(extract_text_folder)
                print(extract_text_folder+"directory created!")

            if missing_only:
                extract_text_if_missing(extract_text_folder+'/'+row["Code"]+".txt", pdf_path)
            else:
                extract_text_from_pdf(pdf_path)
        else:
            print("can't extract because no pdf",row["Code"])

In [10]:
extract_all(metadata_df_test) 

../02_Output/00_Extracted-Text/03_National/LKA2directory created!


## <h3> Text Cleaning </h3>

In [91]:
#Text cleaning 

def remove_noise(text):
    cleaned_text = text.replace('\uf0a7', ';')
    cleaned_text = text.replace('\r', '\n')
    cleaned_text = re.sub(r"\n", " ", cleaned_text)  # remove newlines
    cleaned_text = re.sub(r"\s+", " ", cleaned_text)  # replace multiple spaces with a single space
    # cleaned_text = re.sub(r"[^a-zA-Z0-9\s]", "", cleaned_text)  # remove non-alphanumeric characters
    cleaned_text = re.sub(r"http\S+|www\S+|ftp\S+", "", cleaned_text) # remove urls
    return cleaned_text

def remove_punctuation(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    return cleaned_text

def remove_stopwords(text): 
    # nltk.download('stopwords') # only need to run this once
    stop_words = set(stopwords.words("english"))
    filtered_text = [word for word in text.split() if word.lower() not in stop_words]
    return " ".join(filtered_text)

def convert_to_lowercase(text): # reduce words to their root forms
    return text.lower()



# def lemmatize_text(text):
#     # nltk.download('wordnet') only need to run this once
#     lemmatizer = WordNetLemmatizer() # tokenize the input text into words
#     words = nltk.word_tokenize(text) # lemmatize each word and collect the results in a list

#     original_words = []
#     lemmatized_words = []
#     for word in words:
#         original_words.append(word)
#         lemmatized_word = lemmatizer.lemmatize(word)
#         lemmatized_words.append(lemmatized_word)
#         # if word != lemmatized_word: # keep track of lemmatized words
#         #     print(f"Word '{word}' changed to '{lemmatized_word}'")

#     lemmatized_text = " ".join(lemmatized_words)
#     return lemmatized_text




In [92]:
def clean_text(extract_text_path,clean_text_path):
    
    with open(extract_text_path, 'r') as file:
        extract_text=file.read()
    
    # Remove unwanted characters, whitespace, etc.
    # Regular expressions can be helpful here
    # Additional cleaning steps as required
    
    clean_text = remove_noise(extract_text)
    # clean_text = remove_punctuation(text)
    # clean_text = remove_stopwords(text)
    # clean_text = convert_to_lowercase(text)
    # clean_text = lemmatize_text(text)
    
    with open(clean_text_path, 'w') as file:
        file.write(clean_text)

In [93]:
def clean_text_if_missing(extract_text_path,clean_text_path):
    
    if not os.path.exists(clean_text_path):
        ##extract text if it is not already present
        clean_text(extract_text_path,clean_text_path)

    else:
        print(clean_text_path,"already exists")

In [96]:
#Run the clean function for all
def clean_all(metadata_df,missing_only=True):
    
    for index, row in metadata_df.iterrows():

        folder="03_National/"+row["Code"].split("-")[0]## update this for more docs
        
        clean_text_folder="../02_Output/01_Cleaned-Text/"+folder
        extract_text_folder="../02_Output/00_Extracted-Text/"+folder
        
        extract_text_path=extract_text_folder+'/'+row["Code"]+".txt"
        clean_text_path=clean_text_folder+'/'+row["Code"]+"-clean.txt"
        
        if os.path.exists(extract_text_path):
            
            if not os.path.exists(clean_text_folder):
                # Create the directory
                os.makedirs(clean_text_folder)
                print(clean_text_folder+"directory created!")

            if missing_only:
                clean_text_if_missing(extract_text_path,clean_text_path)
            else:
                clean_text(extract_text_path,clean_text_path)
        else:
            print("can't clean because no extract",row["Code"])

In [97]:
clean_all(metadata_df)

../02_Output/01_Cleaned-Text/03_National/AFG/AFG-CPD-2014-EN-clean.txt already exists
../02_Output/01_Cleaned-Text/03_National/AFG/AFG-CPD-2014-FR-clean.txt already exists
../02_Output/01_Cleaned-Text/03_National/AFG/AFG-CPD-2014-SP-clean.txt already exists
../02_Output/01_Cleaned-Text/03_National/AFG/AFG-NEP-2015-EN-clean.txt already exists
../02_Output/01_Cleaned-Text/03_National/AFG/AFG-NREP-41365-EN-clean.txt already exists
../02_Output/01_Cleaned-Text/03_National/AFG/AFG-NREP-2015-EN-clean.txt already exists
../02_Output/01_Cleaned-Text/03_National/AFG/AFG-NRES-2017-PR-clean.txt already exists
../02_Output/01_Cleaned-Text/03_National/AFG/AFG-NRER-2017-EN-clean.txt already exists
../02_Output/01_Cleaned-Text/03_National/AFG/AFG-NEPro-2022-EN-clean.txt already exists
../02_Output/01_Cleaned-Text/03_National/AFG/AFG-NES-2007-EN-clean.txt already exists
../02_Output/01_Cleaned-Text/03_National/AFG/AFG-NEEP-2016-EN-clean.txt already exists
../02_Output/01_Cleaned-Text/03_National/AFG/A

<h3> convert final cleaned text and metadata to jsons </h3>

In [138]:
def create_document_json(json_path,clean_text_path,row):
    
    with open(clean_text_path, 'r') as file:
        text=file.read()
    
    document_json = {
        'Code': str(row['Code']),
        'Status': row['Status'],
        'Country Name': row['Country Name'],
        'Country Code': str(row['Country Code']),
        'Category': row['Category'],
        'KeyWord to Search': row['KeyWord to Search'],
        'Document Title': row['Title'],
        'Exists?': row['Exists?'],
        'Category': row['Category'],
        'Publication Date': str(row['Publication Date']),
        'Publication Year': str(row['Publication Date']),
        'Start Year': str(row['Start Year']),
        'End Year': str(row['End Year']),
        'Language': row['Language'],
        'Link': row['Link'],
        'Content': text  # Extracted text content from PDF
    }


    # Write the dictionary to a JSON file
    with open(json_path, 'w') as jsonfile:
        json.dump(document_json, jsonfile)
 

In [139]:
def jsonify_if_missing(json_path,clean_text_path,row):
    
    if not os.path.exists(json_path):
        ##extract text if it is not already present
        create_document_json(json_path,clean_text_path,row)

    else:
        print(json_path,"already exists")

In [140]:
def jsonify_all(metadata_df,missing_only=True):
    
    metadata_df=metadata_df.fillna(np.nan).replace([np.nan], [None])##fixes missing so json works

    for index, row in metadata_df.iterrows():

        folder="03_National/"+row["Code"].split("-")[0]## update this for more docs
        
        clean_text_folder="../02_Output/01_Cleaned-Text/"+folder ##later update this to the manual folder
        json_folder="../02_Output/03_Pdf-Jsons/"+folder
        
        clean_text_path=clean_text_folder+'/'+row["Code"]+"-clean.txt"
        json_path=json_folder+'/'+row["Code"]+".json"
        
        if os.path.exists(clean_text_path):
            
            if not os.path.exists(json_folder):
                # Create the directory
                os.makedirs(json_folder)
                print(json_folder+" directory created!")

            if missing_only:
                jsonify_if_missing(json_path,clean_text_path,row)
            else:
                create_document_json(json_path,clean_text_path,row)
        else:
            print("can't jsonify because no clean text",row["Code"])

In [141]:
jsonify_all(metadata_df)

../02_Output/03_Pdf-Jsons/03_National/AFG directory created!
../02_Output/03_Pdf-Jsons/03_National/ALB directory created!
../02_Output/03_Pdf-Jsons/03_National/ALB/ALB-NREP-2021-EN.json already exists
can't jsonify because no clean text ALB-NGP-2016-EN
can't jsonify because no clean text ALB-NREDS-2005-EN
can't jsonify because no clean text ALB-NREEP-2013-EN
../02_Output/03_Pdf-Jsons/03_National/DZA directory created!
can't jsonify because no clean text DZA-NREP-43952-EN
../02_Output/03_Pdf-Jsons/03_National/AGO directory created!
can't jsonify because no clean text AGO-NREP-2023-FR
can't jsonify because no clean text AGO-SEP-2023-EN
../02_Output/03_Pdf-Jsons/03_National/ARG directory created!
../02_Output/03_Pdf-Jsons/03_National/ARM directory created!
